In [3]:
"""
FP Growth 演算法程式碼來自，經過部分微調
作者：ls秦
出處：http://www.cnblogs.com/lsqin/
"""
# FP tree結構
import pandas as pd


class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue  # 節點元素名稱，在構造時初始化爲給定值
        self.count = numOccur   # 出現次數，在構造時初始化爲給定值
        self.nodeLink = None   # 指向下一個相似節點的指針，默認爲None
        self.parent = parentNode   # 指向父節點的指針，在構造時初始化爲給定值
        self.children = {}  # 指向子節點的字典，以子節點的元素名稱爲鍵，指向子節點的指針爲值，初始化爲空字典

    # 增加節點的出現次數值
    def inc(self, numOccur):
        self.count += numOccur

    # 輸出節點和子節點的FP樹結構
    def disp(self, ind=1):
        print(' ' * ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind + 1)

# =======================================================構建FP樹==================================================

# 對不是第一個出現的節點，更新頭指針塊。就是添加到相似元素鏈表的尾部


def updateHeader(nodeToTest, targetNode):
    while (nodeToTest.nodeLink != None):
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

# 根據一個排序過濾後的頻繁項更新FP樹


def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:
        # 有該元素項時計數值+1
        inTree.children[items[0]].inc(count)
    else:
        # 沒有這個元素項時創建一個新節點
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        # 更新頭指針表或前一個相似元素項節點的指針指向新節點
        if headerTable[items[0]][1] == None:  # 如果是第一次出現，則在頭指針表中增加對該節點的指向
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])

    if len(items) > 1:
        # 對剩下的元素項迭代調用updateTree函數
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

# 主程序。創建FP樹。dataSet爲事務集，爲一個字典，鍵爲每個事物，值爲該事物出現的次數。minSup爲最低支持度


def createTree(dataSet, minSup=1):
    # 第一次遍歷數據集，創建頭指針表
    headerTable = {}
    for trans in dataSet:
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    # 移除不滿足最小支持度的元素項
    keys = list(headerTable.keys())  # 因爲字典要求在迭代中不能修改，所以轉化爲列表
    for k in keys:
        if headerTable[k] < minSup:
            del (headerTable[k])
    # 空元素集，返回空
    freqItemSet = set(headerTable.keys())
    if len(freqItemSet) == 0:
        return None, None
    # 增加一個數據項，用於存放指向相似元素項指針
    for k in headerTable:
        headerTable[k] = [headerTable[k], None]  # 每個鍵的值，第一個爲個數，第二個爲下一個節點的位置
    retTree = treeNode('Null Set', 1, None)  # 根節點
    # 第二次遍歷數據集，創建FP樹
    for tranSet, count in dataSet.items():
        localD = {}  # 記錄頻繁1項集的全局頻率，用於排序
        for item in tranSet:
            if item in freqItemSet:   # 只考慮頻繁項
                localD[item] = headerTable[item][0]  # 注意這個[0]，因爲之前加過一個數據項
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(
                localD.items(), key=lambda p: p[1], reverse=True)]  # 排序
            updateTree(orderedItems, retTree, headerTable, count)  # 更新FP樹
    return retTree, headerTable

# =================================================查找元素條件模式基===============================================

# 直接修改prefixPath的值，將當前節點leafNode添加到prefixPath的末尾，然後遞歸添加其父節點。
# prefixPath就是一條從treeNode（包括treeNode）到根節點（不包括根節點）的路徑


def ascendTree(leafNode, prefixPath):
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)

# 爲給定元素項生成一個條件模式基（前綴路徑）。basePet表示輸入的頻繁項，treeNode爲當前FP樹中對應的第一個節點
# 函數返回值即爲條件模式基condPats，用一個字典表示，鍵爲前綴路徑，值爲計數值。


def findPrefixPath(basePat, treeNode):
    condPats = {}  # 存儲條件模式基
    while treeNode != None:
        prefixPath = []  # 用於存儲前綴路徑
        ascendTree(treeNode, prefixPath)  # 生成前綴路徑
        if len(prefixPath) > 1:
            # 出現的數量就是當前葉子節點的數量
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink  # 遍歷下一個相同元素
    return condPats

# =================================================遞歸查找頻繁項集===============================================
# 根據事務集獲取FP樹和頻繁項。
# 遍歷頻繁項，生成每個頻繁項的條件FP樹和條件FP樹的頻繁項
# 這樣每個頻繁項與他條件FP樹的頻繁項都構成了頻繁項集

# inTree和headerTable是由createTree()函數生成的事務集的FP樹。
# minSup表示最小支持度。
# preFix請傳入一個空集合（set([])），將在函數中用於保存當前前綴。
# freqItemList請傳入一個空列表（[]），將用來儲存生成的頻繁項集。


def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    # 對頻繁項按出現的數量進行排序進行排序
    # 返回重新排序的列表。每個元素是一個元組，[（key,[num,treeNode],()）
    sorted_headerTable = sorted(headerTable.items(), key=lambda p: p[1][0])
    bigL = [v[0] for v in sorted_headerTable]  # 獲取頻繁項
    for basePat in bigL:
        newFreqSet = preFix.copy()  # 新的頻繁項集
        newFreqSet.add(basePat)     # 當前前綴添加一個新元素
        freqItemList.append(newFreqSet)  # 所有的頻繁項集列表
        # 獲取條件模式基。就是basePat元素的所有前綴路徑。它像一個新的事務集
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
        myCondTree, myHead = createTree(condPattBases, minSup)  # 創建條件FP樹

        if myHead != None:
            # 用於測試
            #print('conditional tree for:', newFreqSet)
            # myCondTree.disp()
            mineTree(myCondTree, myHead, minSup,
                     newFreqSet, freqItemList)  # 遞歸直到不再有元素

# 生成數據集


def loadSimpDat(data):
    simpDat = data
    return simpDat

# 將數據集轉化爲目標格式


def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict


def fpGrowth(data, minSup=3):
    simpDat = loadSimpDat(data)  # 加載數據集
    initSet = createInitSet(simpDat)  # 轉化爲符合格式的事務集
    myFPtree, myHeaderTab = createTree(initSet, minSup)  # 形成FP樹
    # myFPtree.disp()  # 打印樹
    freqItems = []  # 用於存儲頻繁項集
    mineTree(myFPtree, myHeaderTab, minSup, set([]), freqItems)  # 獲取頻繁項集
    return freqItems

In [1]:
import copy
# 將小集合與大集合合併 ex [{1,2},{1,3},{1,2,3},{5}] => [{5},{1,2,3}]


def mergeFreqItems(freqItems):
    # 將FreqItems按照集合的len進行排序，由小而大
    sotedFreqItems = sorted(freqItems, key=lambda i: len(i))
    dynamicFreqItem = copy.deepcopy(
        sotedFreqItems)  # 排序後FreqItems的複製，會在運算過程變動內容
    for i in range(len(sotedFreqItems)):
        Is_longer_set_existed = False
        for j in range(len(dynamicFreqItem)):
            if sotedFreqItems[i] == dynamicFreqItem[j]:
                del_indx = j  # 記錄下在dynamicFreqItem裡與sotedFreqItems[i]相等值的index
                continue
            intersetion = sotedFreqItems[i] & dynamicFreqItem[j]
            # 相等代表找到>sotedFreqItems[i]的集合
            if intersetion == sotedFreqItems[i]:
                Is_longer_set_existed = True
        # 當較大的集合存在，則不須保留dynamicFreqItem中的sotedFreqItems[i]
        if Is_longer_set_existed == True:
            del dynamicFreqItem[del_indx]
    mergeResult = dynamicFreqItem
    return mergeResult

In [75]:
# 若文章包含fliterItem，將其從文本中移除
def fliterArticles(rawArticles, fliterItems):
    fliteredArticles = []
    for article in rawArticles:
        preFliteredSet = set(article)
        fliteredSet = set(article)
        for item in fliterItems:
            intersetion = item & set(article)
            # 若交集與fliter相等，代表fliter set所有的元素都存在article中
            if intersetion == item:
                fliteredSet = set(article)-item  # 提取過濾後的article
            fliteredSet = fliteredSet & preFliteredSet  # 一個article可與多個fliter相符，因此全部都要and
            preFliteredSet = fliteredSet
        fliteredArticles.append(list(fliteredSet))
    return fliteredArticles

In [92]:
# 讀取資料(斷詞結果)、FP Growth
data = pd.read_pickle('FPM_input81000.pkl')
rawArticles = data.text
firstResult = fpGrowth(rawArticles, minSup=23000)
# print(firstResult)

In [93]:
mergeFreqItems(firstResult)

[{'事項', '敘明', '本', '發生'},
 {'事項', '公司', '日期', '本'},
 {'事項', '其他', '日期', '本'},
 {'事項', '公司', '其他', '本', '發生'},
 {'事項', '公司', '其他', '應', '敘明', '請'},
 {'事項', '其他', '應', '敘明', '無', '發生'},
 {'事項', '公司', '其他', '應', '敘明', '發生'},
 {'事項', '其他', '應', '敘明', '日期', '無'},
 {'事項', '公司', '其他', '應', '敘明', '日期'},
 {'事項', '公司', '其他', '應', '敘明', '日', '本'},
 {'事項', '公司', '其他', '應', '敘明', '本', '無'}]

In [72]:
# fliterItems=mergeFreqItems(firstResult) #會比較快
fliterItems = firstResult  # 這個方式會過濾調比較多詞
# print(fliterItems)

In [76]:
fliteredArticles = fliterArticles(rawArticles, fliterItems)

In [77]:
# test
print(set(rawArticles[0]) - set(fliteredArticles[0]))

{'無', '日期', '其他', '發生', '日', '應', '公司', '本', '敘明', '事項'}


In [78]:
# 第二次FP Growth
secondResult = fpGrowth(fliteredArticles, minSup=5000)

In [85]:
FPMresult = mergeFreqItems(secondResult)
# print(FPMresult)

[{'用途'}, {'且'}, {'準則'}, {'利益'}, {'為止'}, {'資產'}, {'影響'}, {'主管'}, {'將'}, {'未'}, {'年度'}, {'比率'}, {'法人'}, {'董事會', '種類'}, {'關係人', '關係'}, {'關係', '目的'}, {'目的', '為'}, {'限制', '不'}, {'否', '原因'}, {'否', '不'}, {'查閱', '訊息'}, {'通過', '同意'}, {'董事會', '同意'}, {'近期', '元'}, {'財務', '近期'}, {'處分', '取得'}, {'對象', '年'}, {'對象', '條件'}, {'占', '金額'}, {'財務', '占'}, {'占', '發行'}, {'占', '元'}, {'意見', '為'}, {'最', '財務'}, {'最', '元'}, {'董事會', '授權'}, {'總金額', '為'}, {'是否', '不'}, {'是否', '年'}, {'通過', '交易'}, {'交易', '金額'}, {'價格', '交易'}, {'交易', '董事'}, {'交易', '元'}, {'交易', '年'}, {'獨立', '董事'}, {'係', '每'}, {'係', '有'}, {'係', '元'}, {'名稱', '係'}, {'係', '事實'}, {'係', '年'}, {'預計', '為'}, {'次', '預計'}, {'預計', '金額'}, {'調整', '者'}, {'資金', '金額'}, {'已', '不'}, {'取得', '事實'}, {'取得', '年'}, {'承認', '取得'}, {'次', '取得'}, {'通過', '取得'}, {'取得', '董事'}, {'該', '金額'}, {'該', '不'}, {'第', '項目'}, {'財務', '項目'}, {'同', '不'}, {'同', '原因'}, {'股份', '科技'}, {'有限公司', '科技'}, {'董事會', '現金'}, {'決議', '現金'}, {'現金', '元'}, {'不', '條件'}, {'期間', '內'}, {'關係', '內'}, {'並', '內'}, {'者', '內'}, {'其',

In [144]:
# set轉list
import pandas as pd
result = []
for item in secondResult:
    result.append(list(item))

# list轉dataframe
result_dict = {'frequentPattern': result}
df = pd.DataFrame(result_dict)
print("Shape:", result_df.shape)

# 儲存成plk
df.to_pickle('FPM_result.plk')

Shape: (98313, 1)


In [145]:
# 讀取與測試
pickle_df = pd.read_pickle('FPM_result.plk')
df.equals(pickle_df)

In [ ]:
#以下為結果測試

In [86]:
def searchKeyword(freqItems, keyword):
    result = []
    for item in freqItems:
        if keyword in item:
            result.append(item)
    return result

In [87]:
searchKeyword(FPMresult, '報表')

[{'報表', '年'},
 {'報表', '有限公司'},
 {'報表', '股'},
 {'報表', '發行', '財務'},
 {'報表', '財務', '通過'},
 {'元', '公開', '報表'},
 {'公開', '報表', '財務'},
 {'報表', '財務', '金額'},
 {'元', '報表', '金額'},
 {'事實', '元', '報表'},
 {'事實', '報表', '財務'},
 {'元', '報表', '財務'},
 {'一', '報表', '財務'},
 {'公開', '名稱', '報表', '關係'},
 {'名稱', '報表', '財務', '關係'},
 {'元', '名稱', '報表', '關係'},
 {'事實', '名稱', '報表', '金額', '關係'}]

In [88]:
searchKeyword(FPMresult, '財務')

[{'財務', '近期'},
 {'占', '財務'},
 {'最', '財務'},
 {'財務', '項目'},
 {'財務', '非'},
 {'三', '財務'},
 {'內容', '財務'},
 {'承認', '財務'},
 {'證券', '財務'},
 {'相關', '財務'},
 {'期間', '財務'},
 {'前', '財務'},
 {'財務', '重要'},
 {'有', '財務'},
 {'並', '財務'},
 {'每', '財務'},
 {'子公司', '財務'},
 {'公告', '財務'},
 {'為', '財務'},
 {'名稱', '該', '財務'},
 {'營業', '盈餘', '財務'},
 {'公開', '財務', '資訊'},
 {'事實', '財務', '資訊'},
 {'事實', '業務', '財務'},
 {'一', '營業', '財務'},
 {'決議', '營業', '財務'},
 {'所', '條', '財務'},
 {'所', '財務', '金額'},
 {'不', '所', '財務'},
 {'名稱', '所', '財務'},
 {'報表', '發行', '財務'},
 {'報表', '財務', '通過'},
 {'公開', '報表', '財務'},
 {'報表', '財務', '金額'},
 {'事實', '報表', '財務'},
 {'元', '報表', '財務'},
 {'一', '報表', '財務'},
 {'條', '說明', '財務'},
 {'其', '者', '財務'},
 {'其', '董事', '財務'},
 {'事實', '第', '財務'},
 {'原因', '營運', '財務'},
 {'情形', '營運', '財務'},
 {'營運', '者', '財務'},
 {'公開', '營運', '財務'},
 {'事實', '營運', '財務'},
 {'名稱', '發行', '財務'},
 {'發行', '財務', '金額'},
 {'元', '發行', '財務'},
 {'次', '發行', '財務'},
 {'名稱', '次', '財務'},
 {'次', '財務', '金額'},
 {'次', '財務', '關係'},
 {'次', '董事會', '財務'},
 {'元', '次

In [89]:
searchKeyword(FPMresult, '子公司')

[{'子公司', '金額'},
 {'子公司', '為'},
 {'子公司', '財務'},
 {'公開', '子公司', '關係'},
 {'公開', '名稱', '子公司'},
 {'子公司', '年', '月'},
 {'事實', '子公司', '月', '關係'},
 {'名稱', '子公司', '月', '關係'},
 {'事實', '名稱', '子公司', '月'},
 {'事實', '因應', '子公司', '年'},
 {'事實', '子公司', '年', '有限公司', '相互'},
 {'事實', '子公司', '年', '相互', '適用'},
 {'不', '子公司', '年', '相互', '適用'},
 {'子公司', '年', '相互', '適用', '關係'},
 {'名稱', '子公司', '年', '相互', '適用'},
 {'名稱', '子公司', '年', '有限公司', '相互', '關係'},
 {'事實', '元', '名稱', '子公司', '有限公司', '關係'},
 {'事實', '名稱', '子公司', '年', '有限公司', '股份', '關係'},
 {'不', '名稱', '子公司', '年', '有限公司', '適用', '關係'},
 {'事實', '名稱', '子公司', '年', '持股', '有限公司', '關係'},
 {'不', '事實', '名稱', '子公司', '年', '有限公司', '關係'},
 {'不', '事實', '名稱', '子公司', '年', '持股', '相互', '關係'},
 {'事實', '名稱', '子公司', '年', '持股', '比例', '相互', '關係'},
 {'不', '事實', '名稱', '子公司', '年', '持股', '比例', '適用', '關係'},
 {'不',
  '事實',
  '名稱',
  '因應',
  '子公司',
  '持股',
  '措施',
  '有限公司',
  '比例',
  '相互',
  '緣由',
  '股份',
  '輸入',
  '適用',
  '關係'}]

In [95]:
searchKeyword(FPMresult, '營運')

[{'營運', '重要'},
 {'營運', '金額'},
 {'為', '營運'},
 {'條', '營運'},
 {'年', '營運'},
 {'元', '營運'},
 {'次', '營運', '發行'},
 {'次', '營運', '董事會'},
 {'不', '營運', '適用'},
 {'名稱', '營運', '關係'},
 {'原因', '營運', '者'},
 {'原因', '營運', '財務'},
 {'情形', '營運', '財務'},
 {'情形', '營運', '者'},
 {'營運', '者', '財務'},
 {'事實', '公開', '營運'},
 {'公開', '營運', '財務'},
 {'事實', '營運', '財務'}]

In [94]:
searchKeyword(FPMresult, '並')

[{'並', '內'},
 {'並', '價格'},
 {'並', '相關'},
 {'並', '五'},
 {'並', '處理'},
 {'並', '說明'},
 {'並', '財務'},
 {'並', '條'},
 {'並', '決議'},
 {'並', '持股'},
 {'並', '比例'},
 {'並', '通過'},
 {'並', '董事'},
 {'並', '情形'},
 {'並', '年', '期間'},
 {'並', '方式', '發行'},
 {'並', '其', '為'},
 {'並', '其', '次'},
 {'並', '其', '董事會'},
 {'並', '每', '董事會'},
 {'並', '次', '每'},
 {'並', '元', '每'},
 {'並', '年', '每'},
 {'並', '公開', '年'},
 {'一', '並', '年'},
 {'一', '並', '董事會'},
 {'並', '名稱', '金額'},
 {'並', '年', '金額'},
 {'並', '元', '金額'},
 {'並', '為', '者'},
 {'並', '次', '者'},
 {'並', '者', '董事會'},
 {'並', '有限公司', '股份'},
 {'並', '有限公司', '關係'},
 {'並', '年', '有限公司'},
 {'並', '事實', '有限公司'},
 {'並', '元', '為'},
 {'並', '事實', '為'},
 {'並', '名稱', '為'},
 {'並', '公告', '名稱'},
 {'並', '事實', '公告'},
 {'並', '元', '關係'},
 {'並', '元', '次'},
 {'並', '元', '董事會'},
 {'並', '事實', '元'},
 {'並', '元', '名稱'},
 {'不', '並', '月'},
 {'並', '月', '董事會'},
 {'並', '年', '月'},
 {'並', '年', '方式', '有'},
 {'並', '方式', '有', '董事會'},
 {'不', '並', '方式', '次'},
 {'並', '年', '方式', '次'},
 {'並', '方式', '次', '董事會'},
 {'不', '並

In [1]:
# 萃取文本中的freqItems(關鍵字)
def extractKeywords(rawArticles, freqItems):
    articleKeywords = []
    for article in rawArticles:
        preFliteredSet = set(article)
        fliteredSet = set(article)
        for item in freqItems:
            item = set(item)
            intersetion = item & set(article)
            # 若交集與fliter相等，代表fliter set所有的元素都存在article中
            if intersetion == item:  
                fliteredSet = set(article)-item  # 提取過濾後的article
            fliteredSet = fliteredSet & preFliteredSet  # 一個article可與多個fliter相符，因此全部都要and
            preFliteredSet = fliteredSet
        keywords = set(article) - fliteredSet  # 全部 - 過濾掉這些詞的部分 = 要萃取的詞
        articleKeywords.append(list(keywords))
    return articleKeywords

In [3]:
import pandas as pd
data = pd.read_pickle('FPM_result.plk')
freqItems = data.frequentPattern

data = pd.read_pickle('FPM_input81000.pkl')
rawArticles = data.text
extractAticles = extractKeywords(rawArticles[:20], freqItems)

In [4]:
# 可以測試6、10 一個反面例子、一個正面例子
testInd = 6

In [5]:
# 驗證結果正確性，驗證一下資料是否是同一筆的結果
set(extractAticles[testInd]) & set(
    rawArticles[testInd]) == set(extractAticles[testInd])

True

In [6]:
print('extract result length:', len(extractAticles[testInd]), '\n')
print('origin data length:', len(rawArticles[testInd]), '\n')

extract result length: 79 

origin data length: 421 



In [7]:
print('原本的內容:\n', set(rawArticles[testInd]))

原本的內容:
 {'非', '處分', '規', '無', 'ＸＸ', '年', '係人', '低', '自然人', '者', '時', '同', '參考', '有', '意見', '地段', '同意', '小段', '人', '總價', '月', '限制', '條款', '為', '關係人', '最近', '段', '結果', '估價', '移轉', '限定', '條件', '交易', '承認', '其他', '得', '利益', '發生日', '發行', '支付', '且', '價格', '面積', '內', '新台幣', '市場', '重要', '需', '樓', '內湖區', '對象', '每', '公告', '比價', '經紀人', '如', '議價', '董事會', '民國', '事實', '證書', '專業', '招標', '不', '相對', '估價師', '中國', '審計', '相對人', '折合', '異議', '尚', '決策', '免', '名稱', '契約', '否', '坐落', '租賃', '敘明', '適用', '通過', '付款', '所有人', '特定', '特殊', '坪', '開業', '電子', '用途', '數量', '總金額', '會計師', '約定', '平方公尺', '表示', '巷號樓', '日', '周邊', '是否', '係', '本', '原因', '監察人', '北區', '應', '委員會', '處理', '說明', '第十六', '定', '金額', '預計', '並', '當時', '第十七', '評估', '相互', '關', '其', '報告', '決定', '有限公司', '前', '含', '重大', '次', '屬', '標的', '經紀', '資產', '單位', '揭露', '公司', '依據', '交付', '列表', '情形', '使用權', '取得', '姓名', '項', '性質', '台北市', '方式', '曾', '認列', '日期', '營運', '五', '所有權人', '準則', '股份', '董事', '條', '行愛路', '計', '事項', '字號', '尚未', '期間', '土地', '所', '損失', '公開', '關係', '行情', '不動產',

In [8]:
print("根據frequent pattern 萃取出來的結果:\n", set(extractAticles[testInd]))

根據frequent pattern 萃取出來的結果:
 {'非', '處分', '年', '自然人', '者', '時', '同', '有', '意見', '同意', '月', '限制', '條款', '為', '關係人', '條件', '交易', '承認', '利益', '發行', '且', '價格', '內', '新台幣', '重要', '對象', '每', '公告', '如', '董事會', '事實', '不', '審計', '名稱', '否', '適用', '通過', '用途', '總金額', '會計師', '是否', '係', '原因', '監察人', '委員會', '處理', '說明', '金額', '預計', '並', '相互', '其', '報告', '決定', '有限公司', '前', '含', '重大', '次', '屬', '資產', '情形', '取得', '姓名', '方式', '營運', '五', '準則', '股份', '董事', '條', '期間', '所', '公開', '關係', '目的', '事務所', '元', '規定'}


In [9]:
print('過濾掉的詞:\n', set(rawArticles[testInd]) - set(extractAticles[testInd]))

過濾掉的詞:
 {'規', '無', 'ＸＸ', '係人', '低', '參考', '地段', '小段', '人', '總價', '最近', '段', '結果', '估價', '移轉', '限定', '其他', '得', '發生日', '支付', '面積', '市場', '需', '樓', '內湖區', '經紀人', '比價', '議價', '民國', '證書', '專業', '招標', '相對', '估價師', '中國', '相對人', '折合', '異議', '尚', '決策', '免', '契約', '坐落', '租賃', '敘明', '付款', '所有人', '特定', '特殊', '坪', '開業', '電子', '數量', '約定', '平方公尺', '表示', '巷號樓', '日', '周邊', '本', '北區', '應', '第十六', '定', '當時', '第十七', '評估', '關', '標的', '經紀', '單位', '揭露', '公司', '依據', '交付', '列表', '使用權', '項', '性質', '台北市', '曾', '認列', '日期', '所有權人', '行愛路', '計', '事項', '字號', '尚未', '土地', '損失', '行情', '不動產', '選定', '具體', '費用', '人間', '差異', '台中市', '標的物', '租金', '遞延'}
